# UFC 경기 승패 예측 (모든 특성 사용 최종본)

## 1. 라이브러리 임포트 및 데이터 로드

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

In [53]:
df = pd.read_csv('UFC.csv')

## 2. 전처리 및 특성 공학

In [54]:
# [전처리] 불필요 컬럼 제거 및 결측치 처리
df.dropna(subset=['winner', 'r_dob', 'b_dob', 'r_stance', 'b_stance', 'r_height', 'b_height'], inplace=True)
df['r_reach'].fillna(df['r_height'], inplace=True)
df['b_reach'].fillna(df['b_height'], inplace=True)
num_cols = df.select_dtypes(include=np.number).columns

# [특성 공학] 타겟 변수 및 파생 변수 생성
df['winner_is_red'] = (df['winner'] == df['r_name']).astype(int)
df['r_dob'] = pd.to_datetime(df['r_dob'])
df['b_dob'] = pd.to_datetime(df['b_dob'])
df['r_age'] = (pd.to_datetime('today') - df['r_dob']).dt.days / 365.25
df['b_age'] = (pd.to_datetime('today') - df['b_dob']).dt.days / 365.25
df.drop(['r_dob', 'b_dob'], axis=1, inplace=True)

# 차이 특성
df['age_diff'] = df['r_age'] - df['b_age']
df['height_diff'] = df['r_height'] - df['b_height']
df['reach_diff'] = df['r_reach'] - df['b_reach']
df['wins_diff'] = df['r_wins'] - df['b_wins']
df['losses_diff'] = df['r_losses'] - df['b_losses']
df['splm_diff'] = df['r_splm'] - df['b_splm']
df['str_acc_diff'] = df['r_str_acc'] - df['b_str_acc']
df['sapm_diff'] = df['r_sapm'] - df['b_sapm']
df['str_def_diff'] = df['r_str_def'] - df['b_str_def']
df['td_avg_diff'] = df['r_td_avg'] - df['b_td_avg']
df['td_acc_diff'] = df['r_td_avg_acc'] - df['b_td_avg_acc']
df['td_def_diff'] = df['r_td_def'] - df['b_td_def']
df['sub_avg_diff'] = df['r_sub_avg'] - df['b_sub_avg']

# 비율 및 복합 특성
# ratio features (0~1 scaled)
df['sig_str_ratio'] = df['r_splm'] / (df['r_splm'] + df['b_splm'] + 1e-6)
df['td_ratio'] = df['r_td_avg'] / (df['r_td_avg'] + df['b_td_avg'] + 1e-6)
df['str_acc_ratio'] = df['r_str_acc'] / (df['r_str_acc'] + df['b_str_acc'] + 1e-6)
df['td_acc_ratio'] = df['r_td_avg_acc'] / (df['r_td_avg_acc'] + df['b_td_avg_acc'] + 1e-6)

# win ratio features
df['r_win_ratio'] = df['r_wins'] / (df['r_wins'] + df['r_losses'] + 1e-6)
df['b_win_ratio'] = df['b_wins'] / (df['b_wins'] + df['b_losses'] + 1e-6)
df['win_ratio_diff'] = df['r_win_ratio'] - df['b_win_ratio']

# ---------- 추가 파생 특성 ----------
# BMI
df['r_bmi'] = df['r_weight'] / ((df['r_height'] / 100) ** 2 + 1e-6)
df['b_bmi'] = df['b_weight'] / ((df['b_height'] / 100) ** 2 + 1e-6)
df['bmi_diff'] = df['r_bmi'] - df['b_bmi']

# Reach / Height 비율
df['r_reach_ht_ratio'] = df['r_reach'] / (df['r_height'] + 1e-6)
df['b_reach_ht_ratio'] = df['b_reach'] / (df['b_height'] + 1e-6)
df['reach_ht_ratio_diff'] = df['r_reach_ht_ratio'] - df['b_reach_ht_ratio']


# 총 경기 수
df['r_total_fights'] = df['r_wins'] + df['r_losses']
df['b_total_fights'] = df['b_wins'] + df['b_losses']
df['total_fights_diff'] = df['r_total_fights'] - df['b_total_fights']

# ---------- 추가 파생 특성 v3 ----------
# 1) 공격 점수: Striking + Grappling 효율 합
df['r_offense_score'] = df['r_str_eff'] + df['r_grap_eff'] if 'r_str_eff' in df.columns else \
                        (df['r_splm'] * df['r_str_acc']) + (df['r_td_avg'] * df['r_td_avg_acc'])
df['b_offense_score'] = df['b_str_eff'] + df['b_grap_eff'] if 'b_str_eff' in df.columns else \
                        (df['b_splm'] * df['b_str_acc']) + (df['b_td_avg'] * df['b_td_avg_acc'])
df['offense_score_diff'] = df['r_offense_score'] - df['b_offense_score']

# 2) 방어 점수: 타격·테이크다운 방어율 평균
df['r_defense_score'] = (df['r_str_def'] + df['r_td_def']) / 2
df['b_defense_score'] = (df['b_str_def'] + df['b_td_def']) / 2
df['defense_score_diff'] = df['r_defense_score'] - df['b_defense_score']

# 3) 순공격 이득(Net Advantage) = 공격 diff + 방어 diff
df['net_advantage'] = df['offense_score_diff'] + df['defense_score_diff']

# 4) 상호작용 특성: 레드의 공격 vs 블루의 방어, 블루의 공격 vs 레드의 방어
df['str_vs_def_diff'] = (df['r_str_acc'] * df['b_str_def']) - (df['b_str_acc'] * df['r_str_def'])

# 5) 공격/방어 스코어 비율 차이
df['off_def_ratio_diff'] = (df['r_offense_score'] / (df['r_defense_score'] + 1e-6)) - \
                           (df['b_offense_score'] / (df['b_defense_score'] + 1e-6))

# 스탠스 조합 (범주형)
df['stance_comb'] = df['r_stance'].astype(str) + '_' + df['b_stance'].astype(str)


## 3. 모델링 준비 (모든 특성 사용)

In [55]:
# 모든 생성된 특성 정의
numerical_features = [col for col in df.columns if df[col].dtype != 'object' and col not in ['winner_is_red', 'winner']]
categorical_features = ['r_stance', 'b_stance', 'stance_comb']

# 최종 데이터셋 정의 및 분할
X = df[numerical_features + categorical_features]
y = df['winner_is_red']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'사용된 총 특성 개수: {X_train.shape[1]}')

사용된 총 특성 개수: 75


## 4-1. 모델 테스트 (LogisticRegression)

In [56]:
# 수치형 전처리: 결측치 평균 대체 + 정규화
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# 범주형 전처리: 결측치 최빈값 대체 + 원핫 인코딩
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 전처리 통합
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

# 모델 정의 (클래스 불균형 고려)
logreg = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

# 파이프라인 구성
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', logreg)
])

# 모델 학습
model_pipeline.fit(X_train, y_train)

# 예측
y_pred = model_pipeline.predict(X_test)

# 예측 확률 추출
y_proba = model_pipeline.predict_proba(X_test)[:, 1]  # 클래스 1(레드 승)의 확률만 추출

# 확률 포함 결과 DataFrame
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Probability': y_proba,
    'Correct': y_test == y_pred
})

print(results_df)
print('LogisticRegression Score: ', accuracy_score(y_test, y_pred))

      Actual  Predicted  Probability  Correct
7346       1          1     0.547887     True
5070       1          1     0.774081     True
7802       1          1     0.825310     True
5532       0          0     0.249534     True
5941       1          0     0.309420    False
...      ...        ...          ...      ...
6837       1          1     0.776406     True
5848       0          0     0.393903     True
7964       1          1     0.661564     True
7628       1          1     0.931696     True
6973       1          1     0.901048     True

[1572 rows x 4 columns]
LogisticRegression Score:  0.710559796437659


## 4-2. 모델 테스트 (RandomForestClassifier)

In [58]:
# 전처리기 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)])

# 클래스 불균형 처리 가중치
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

# 기본 모델 정의
rf = RandomForestClassifier(random_state=42, class_weight='balanced')

# 전처리 + 모델을 하나의 파이프라인으로 구성
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# 모델 학습
model_pipeline.fit(X_train, y_train)

# 예측
y_pred = model_pipeline.predict(X_test)

# 예측 확률 추출
y_proba = model_pipeline.predict_proba(X_test)[:, 1]  # 클래스 1(레드 승)의 확률만 추출

# 확률 포함 결과 DataFrame
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Probability': y_proba,
    'Correct': y_test == y_pred
})

print(results_df)
print('RandomForest Score: ',model_pipeline.score(X_test, y_test))

      Actual  Predicted  Probability  Correct
7346       1          1         0.65     True
5070       1          1         0.79     True
7802       1          1         0.67     True
5532       0          0         0.47     True
5941       1          1         0.79     True
...      ...        ...          ...      ...
6837       1          1         0.78     True
5848       0          0         0.34     True
7964       1          1         0.84     True
7628       1          1         0.77     True
6973       1          1         0.94     True

[1572 rows x 4 columns]
RandomForest Score:  0.7315521628498728


## 4-3. 모델 테스트 (XGBoost)

In [60]:
# 수치형 전처리
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# 범주형 전처리
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 전처리 통합
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

# 기본 모델 정의
rf = XGBClassifier()

# XGBoost 모델 정의
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    scale_pos_weight=y_train.value_counts()[0] / y_train.value_counts()[1],
    random_state=42
)

# 파이프라인 구성
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb_model)
])

# 모델 학습
model_pipeline.fit(X_train, y_train)

# 예측
y_pred = model_pipeline.predict(X_test)
y_proba = model_pipeline.predict_proba(X_test)[:, 1]  # 클래스 1(레드 승) 확률

# 결과 DataFrame
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Probability': y_proba,
    'Correct': y_test == y_pred
})

print(results_df)
print('XGBoost Score: ',model_pipeline.score(X_test, y_test))

      Actual  Predicted  Probability  Correct
7346       1          1     0.584937     True
5070       1          1     0.868597     True
7802       1          1     0.986028     True
5532       0          0     0.431876     True
5941       1          1     0.997850     True
...      ...        ...          ...      ...
6837       1          1     0.803543     True
5848       0          1     0.752012    False
7964       1          1     0.915872     True
7628       1          1     0.980007     True
6973       1          1     0.978543     True

[1572 rows x 4 columns]
XGBoost Score:  0.7410941475826972
